In [1]:
from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input,decode_predictions


In [2]:
model=MobileNet(weights='imagenet',include_top=False,classes=1000)
model.summary()


17227776/17225924 [==============================] - 0s 0us/step
Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128       
_________________________________________________

In [3]:
##now i will create my train dataset 
from google.colab import files

uploaded = files.upload()

Saving Train.zip to Train.zip


In [ ]:
!unzip Train.zip

In [50]:
!ls 'Train (1)'/Aerodactyl


029d0c659dc448f49993648b024e615c.jpg  c562bf7015de45f59332f711bb81e926.jpg
0598db9a1de245db9719661466001511.jpg  d1d381e5f2df42a0973e0251751e1a14.jpg
0598e2f5a108470e9fced4079b1105ca.jpg  d995b3fc3cbe47e4811493dd8f1c4ada.jpg
065e79403c774ea288646043465a393f.jpg  dbbd1c859b8d48878421e058f2d7e26c.jpg
0a15b831d1b947fda035aaf637c58215.jpg  e374261001b84fdbbfb7e2427c0f2c9c.jpg
0e87220a99a54ac296e49ee01cec144e.jpg  e8d4cf4ee37a471bb92ee42e298eb692.jpg
0eab1e1f744340fa87268f2b558c7ccf.jpg  ef7f7eeaa897478bbe1df7b0bb0522cb-10.jpg
11c991617d56476ba4d926a87d7fc684.jpg  ef7f7eeaa897478bbe1df7b0bb0522cb-14.jpg
1727be397509465f9d3dafbf3849a8f1.jpg  ef7f7eeaa897478bbe1df7b0bb0522cb-15.jpg
25adea61e7624a84a8fbe88d7dcbeb6e.jpg  ef7f7eeaa897478bbe1df7b0bb0522cb-16.jpg
25d15a304c354f3e8064249cf704370d.jpg  ef7f7eeaa897478bbe1df7b0bb0522cb-17.jpg
263b3cb088e5448e99335fbda8f73ee7.jpg  ef7f7eeaa897478bbe1df7b0bb0522cb-18.jpg
35933037ff854c2784e5199f6dc8a728.jpg  ef7f7eeaa897478bbe1df7b0bb0522cb-19.jpg
4197

In [53]:

from  pathlib import Path
from random import shuffle
from keras.utils import np_utils
import random
import cv2
import numpy as np
image_data=[]
labels=[]
label_dict={}
p=Path('Train (1)')
dirs=list(p.glob("*"))
index=0
for folder in dirs:
    label=str(folder).split('\\')[-1]
    if label in label_dict.keys():
        pass
    else:
        label_dict.update({label:index})
        index+=1
    for img_path in folder.glob("*.jpg"):
        img=cv2.imread(str(img_path))
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        img=cv2.resize(img,(224,224))
        image_data.append(img)
        labels.append(int(label_dict[label]))
image_data=np.array(image_data)
labels=np.array(labels)
label_dict
combine=list(zip(image_data,labels))
random.shuffle(combine)
image_data[:],labels[:]=zip(*combine)
x=np.array(image_data)
y=np_utils.to_categorical(labels,index+1)
labels

array([0, 9, 0, ..., 8, 1, 4])

In [84]:
labels.shape

(1287,)

In [ ]:
from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input,decode_predictions
from keras.optimizers import Adam
from keras.layers import *
from keras.models import Model
model=MobileNet(weights='imagenet',include_top=True,classes=1000)
model.summary()

In [108]:
av1=GlobalAveragePooling2D()(model.output)
fc1=Dense(256,activation='relu')(av1)
d1=Dropout(0.5)(fc1)
fc2=Dense(index+1,activation='softmax')(d1)
model_new=Model(inputs=model.inputs,outputs=fc2)
model.summary()

ValueError: ignored

In [109]:
adam=Adam(lr=0.0003)
model_new.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])

In [ ]:
for ix in range(len(model_new.layers)):
  print(ix,model_new.layers[ix])
  

#model_new.layers

In [110]:
for ix in range(80):
  model_new.layers[ix].trainable=False

In [ ]:
model_new.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])
model.summary()

In [112]:
hist=model_new.fit(x,y,shuffle=True,validation_split=0.10,batch_size=32,epochs=20)

Epoch 1/20
37/37 [==============================] - 12s 234ms/step - loss: 2.5390 - accuracy: 0.2281 - val_loss: 5.7172 - val_accuracy: 0.1705
Epoch 2/20
37/37 [==============================] - 8s 216ms/step - loss: 1.7248 - accuracy: 0.4386 - val_loss: 3.7271 - val_accuracy: 0.1085
Epoch 3/20
37/37 [==============================] - 8s 212ms/step - loss: 1.5380 - accuracy: 0.5177 - val_loss: 3.0638 - val_accuracy: 0.0853
Epoch 4/20
37/37 [==============================] - 8s 207ms/step - loss: 1.4982 - accuracy: 0.4857 - val_loss: 3.1735 - val_accuracy: 0.1240
Epoch 5/20
37/37 [==============================] - 8s 203ms/step - loss: 1.4044 - accuracy: 0.5393 - val_loss: 3.2947 - val_accuracy: 0.1163
Epoch 6/20
37/37 [==============================] - 7s 201ms/step - loss: 1.3251 - accuracy: 0.5439 - val_loss: 3.1682 - val_accuracy: 0.1240
Epoch 7/20
37/37 [==============================] - 7s 199ms/step - loss: 1.2575 - accuracy: 0.5274 - val_loss: 3.5502 - val_accuracy: 0.0853
Epoch

In [ ]:
####now just creating a resneet here
## you can say a new project#####################################################################################################
from keras.applications.resnet50 import ResNet50
model=ResNet50(weights='imagenet',include_top=False,classes=1000)
model.summary()
av1=GlobalAveragePooling2D()(model.output)
fc1=Dense(256,activation='relu')(av1)
d1=Dropout(0.5)(fc1)
fc2=Dense(index+1,activation='softmax')(d1)
model_new=Model(inputs=model.inputs,outputs=fc2)
model.summary()

In [ ]:
model_new.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])
for ix in range(len(model_new.layers)):
  print(ix,model_new.layers[ix])
  

In [100]:
for ix in range(169):
  model_new.layers[ix].trainable=False

In [101]:
hist2=model_new.fit(x,y,shuffle=True,validation_split=0.10,batch_size=32,epochs=20)

Epoch 1/20
37/37 [==============================] - 19s 358ms/step - loss: 2.7063 - accuracy: 0.2009 - val_loss: 1105.0135 - val_accuracy: 0.0930
Epoch 2/20
37/37 [==============================] - 12s 323ms/step - loss: 2.2208 - accuracy: 0.2146 - val_loss: 27.3900 - val_accuracy: 0.1240
Epoch 3/20
37/37 [==============================] - 12s 334ms/step - loss: 2.1068 - accuracy: 0.2742 - val_loss: 110.2603 - val_accuracy: 0.1395
Epoch 4/20
37/37 [==============================] - 12s 337ms/step - loss: 1.9517 - accuracy: 0.3367 - val_loss: 4.6106 - val_accuracy: 0.1008
Epoch 5/20
37/37 [==============================] - 12s 328ms/step - loss: 1.8878 - accuracy: 0.3735 - val_loss: 3.5153 - val_accuracy: 0.1628
Epoch 6/20
37/37 [==============================] - 12s 323ms/step - loss: 1.7858 - accuracy: 0.4179 - val_loss: 2.5826 - val_accuracy: 0.1240
Epoch 7/20
37/37 [==============================] - 12s 322ms/step - loss: 1.7309 - accuracy: 0.4259 - val_loss: 2.7932 - val_accuracy: 

In [127]:
#####creating a whole new model by seeing these accuracy i m implementing vgg-16
from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input,decode_predictions
from keras.optimizers import Adam
from keras.layers import *
from keras.models import Sequential
from keras.models import Model
model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(224,224,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(256,(3,3),activation='relu'))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(256,(3,3),activation='relu'))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(256,(3,3),activation='relu'))
model.add(GlobalAveragePooling2D())
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(10,activation='softmax'))
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_45 (Conv2D)           (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d_34 (MaxPooling (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 109, 109, 64)      18496     
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 54, 54, 64)        0         
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 52, 52, 64)        36928     
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 50, 50, 128)       73856     
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 25, 25, 128)     

In [130]:
adam=Adam(lr=0.0001)
model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])

In [131]:
hist=model_new.fit(x,y,validation_split=0.10,batch_size=64,epochs=10)

Epoch 1/10
19/19 [==============================] - 7s 370ms/step - loss: 0.7762 - accuracy: 0.5553 - val_loss: 6.2114 - val_accuracy: 0.1240
Epoch 2/10
19/19 [==============================] - 7s 379ms/step - loss: 0.7882 - accuracy: 0.5691 - val_loss: 5.6977 - val_accuracy: 0.1318
Epoch 3/10
19/19 [==============================] - 7s 391ms/step - loss: 0.7706 - accuracy: 0.5708 - val_loss: 7.9377 - val_accuracy: 0.1240
Epoch 4/10
19/19 [==============================] - 8s 401ms/step - loss: 0.7558 - accuracy: 0.5820 - val_loss: 8.2986 - val_accuracy: 0.1163
Epoch 5/10
19/19 [==============================] - 8s 402ms/step - loss: 0.7726 - accuracy: 0.5622 - val_loss: 9.0081 - val_accuracy: 0.1395
Epoch 6/10
19/19 [==============================] - 7s 392ms/step - loss: 0.7736 - accuracy: 0.5613 - val_loss: 9.1473 - val_accuracy: 0.1163
Epoch 7/10
19/19 [==============================] - 7s 384ms/step - loss: 0.7555 - accuracy: 0.5777 - val_loss: 9.0327 - val_accuracy: 0.1240
Epoch 